# Petitie Aprilbeweging 1853

Van prof. dr. Fred van Lieburg een digitale versie verkegen van een petitie tegen de het herstel van de bisschoppelijke hiërarchie in Nederland door paus Pius IX. Deze petitie is was een protest vanui protestantse en conservatieve hoek en staat ook wel bekend als de *Aprilbeweging* https://nl.wikipedia.org/wiki/Aprilbeweging. De beweging richtte zich tevens tegen het kabinet-Thorbecke I, dat de Rooms-Katholieke Kerk geen strobreed in de weg legde, op grond van de scheiding van kerk en staat en de vrijheid van godsdienst.

In de getranscribeerde petitie, geleverd als excelsheet, staan plaatsnamen en provincies. Om een choropleth kaart te kunnen produceren met daarop de ruimtelijk verspreiding van de aantallen die deze petitie hebben ondertekend dienen de plaatsnamen gekoppeld te worden aan historische gemeentegrenzen. 

Voor de historische gemeentegrenzen is gebruik gemaakt van;
Zijdeman, Richard; Mac Gillavry, Edward, 2020, "nlgis-boundary-files with Amsterdam Code", https://hdl.handle.net/10622/URI8O2, IISH Data Collection, V1 
en zijn de gemeentegrenzen van 1853 geselecteerd.

De petitie telt 933 gemeentes / plaatnamen

De gemeentes uit 1853 zijn er in totaal 1209.


Het matchen van de gemeentegrenzen met de petitie vergde een aantal stappen. 

**Stap 1:** Een join op basis van de plaatsnamen in het excelsheet en het 1853 gemeentegrenzen bestand.
*Dit leverde 427 matches op.*

**Stap 2.1:** 
Dit betekent dat er nog 933-427=506 plaatsnamen nog niet gematcht konden worden met de oude gemeentegrenzen. Een van de oorzaken hiervan is dat - ook toen - gemeentes uit meerdere plaatsen konden bestaan. 
Bijvoorbeeld; het plaatsje Holysloot maakte, wanneer uit wordt gegaan van de 1853 gemeentegrenzen, onderdeel uit van de gemeente Ransdorp. In de tweede stap worden dan ook de overbleven plaatsnamen gegeocodeerd op basis van de locatieserver van pdok. Hierbij is de aanname dat de plaatsnamen door de tijd niet zijn veranderd. 

**Stap 2.2:** 
Om vervolgens de plaatsnamen aan de gemeentegrenzen te koppelen is een spatial join uitgevoerd en zijn de aantallen, wanneer deze toebehoren aan eenzelfde 1853 gemeente bij elkaar opgeteld.

**Stap 3:**
Na al deze stappen bleven er nog 56 plaatsnamen over die wel op de petitie staan, maar niet in het gemeentegrenzen of konden worden gegeocodeerd op basis van huidige plaatsnamen. Deze zijn handmatig nagelopen.

In [172]:
933-427

506

In [177]:
# In deze stap worden de benodigde libraries geïmporteerd en functies gedefinieerd om plaatsnamen te geocoderen voor stap 2
import time
import requests
import pandas as pd
import geopandas as gpd


# Panda settings for showing data (this is foremost done to more easily explore the data while processing it)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)



PDOK_URL = "https://api.pdok.nl/bzk/locatieserver/search/v3_1/free"

ALLOWED_TYPES = {"woonplaats", "gemeente"}  # looser: also municipalities

def geocode_pdok_placeonly(query: str, rows: int = 5, timeout: int = 20):
    params = {"q": query, "rows": rows}
    r = requests.get(PDOK_URL, params=params, timeout=timeout)
    r.raise_for_status()
    docs = r.json().get("response", {}).get("docs", [])

    # filter to allowed types
    candidates = [d for d in docs if d.get("type") in ALLOWED_TYPES]
    if not candidates:
        return None, None, None, None, None

    # pick best scoring allowed candidate
    best = max(candidates, key=lambda d: d.get("score", 0))

    label = best.get("weergavenaam")
    score = best.get("score")
    typ = best.get("type")

    centroide_ll = best.get("centroide_ll")  # "POINT(lon lat)"
    if not centroide_ll:
        return None, None, label, score, typ

    inside = centroide_ll.replace("POINT(", "").replace(")", "")
    lon_str, lat_str = inside.split()
    lon, lat = float(lon_str), float(lat_str)

    return lat, lon, label, score, typ


def geocode_list_pdok_placeonly(names, sleep_s: float = 0.05):
    out = []
    for n in names:
        lat, lon, label, score, typ = geocode_pdok_placeonly(n)
        out.append({"query": n, "match_label": label, "type": typ, "lat": lat, "lon": lon, "score": score})
        time.sleep(sleep_s)
    return pd.DataFrame(out)

In [ ]:
folder_path = "E://Dropbox//ONDERNEMEN_MAURICE//opdracht9_Lieburg_atlas//1853_map//"

In [ ]:
# import the gemeentegrenzen uit 1853. 
gem_1853 = gpd.read_file(folder_path+"nl_1853.geojson")

# projecteer gem_1853 naar RD_new 28992
gem_1853 = gem_1853.to_crs(epsg=28992)

In [ ]:
# Omdat sommige plaatsnamen in meerdere provincies voorkomen zullen we voor stap 1 een join uitvoeren op basis van de combinatie plaatsnaam provincie. Echter bevat het bestand nl_1853.geojson geen provincie id. 
# Omdat de provinciegrenzen door de tijd heen ook zijn veranderd is uitgegaan van de oudst vindbare provincie grenzen waarbij zowel de gemeentegrenzen en provinciegrenzen te vinden zijn. 
# De dataset die hiervoor gevonden is afkomstig van; https://www.cbs.nl/nl-nl/dossier/nederland-regionaal/geografische-data/cbs-gebiedsindelingen . Er is gekozen voor 1995 (de oudste)
prov_1995 = gpd.read_file( folder_path+"cbsgebiedsindelingen1995_2015//cbsgebiedsindelingen1995.gpkg"  , layer="provincie_gegeneraliseerd")

In [ ]:
# omdat de geometrien van prov_1995 en gem_1853 niet exact overeen komen. Er is sprake van klein slivers in het digitaliseerproces lukt gem_1853_provincie = gpd.sjoin(gem_1853, prov_1995, how="inner", predicate="within") helaas niet.
# Daarom wordt het gedaan op basis van een intersect en de grootste overlap.
 
# 1): all pairs that intersect
intersect_st1 = gpd.sjoin(
    gem_1853.reset_index(names="left_id"),
    prov_1995.reset_index(names="right_id"),
    how="left",
    predicate="intersects"
)

In [ ]:
# 2) Compute intersection area for each candidate pair
# (merge geometries back in so we can intersect them)
intersect_st2 = intersect_st1.merge(
    prov_1995.reset_index(names="right_id")[["right_id", "geometry"]],
    on="right_id",
    suffixes=("_left", "_right"),
)

In [146]:
intersect_st2 = intersect_st2.set_geometry("geometry_left")

intersect_st2["overlap_area"] = (
    intersect_st2.geometry
        .intersection(intersect_st2["geometry_right"])
        .area
)

In [147]:
# 3) Keep the single best match per left polygon (largest overlap)
best = (
    intersect_st2.sort_values(["left_id", "overlap_area"], ascending=[True, False])
        .drop_duplicates("left_id")
)

# 4) Build final output: left polygons + selected right attributes
# Drop right geometry if you only want left geometry in the result
result = (
    gem_1853.reset_index(names="left_id")
        .merge(
            best.drop(columns=["geometry_right"], errors="ignore"),
            on="left_id",
            how="left",
            suffixes=("", "_rightattrs"),
        )
)

In [149]:
result.shape

(1209, 19)

In [157]:
result["statnaam"].value_counts().sort_index()

Drenthe           33
Friesland         45
Gelderland       120
Groningen         57
Limburg          125
Noord-Brabant    185
Noord-Holland    142
Overijssel        62
Utrecht           97
Zeeland          116
Zuid-Holland     227
Name: statnaam, dtype: int64

In [153]:
flevoland = result[result["statnaam"] == "Flevoland"]

In [154]:
flevoland.head()

,left_id,naam,acode,cbscode,year,geometry,naam_rightattrs,acode_rightattrs,cbscode_rightattrs,year_rightattrs,geometry_left,index_right,right_id,id,statcode,statnaam,jrstatcode,rubriek,overlap_area
763,763,Urk,10783,184,1853,"POLYGON ((169430.005 518700.001, 169080.005 518520.001, 168710.005 518940.001, 169010.005 519340.001, 169790.005 519780.001, 170260.005 519900.001, 170330.005 519560.001, 169820.005 519260.001, 169430.005 518700.001))",Urk,10783,184,1853,"POLYGON ((169430.005 518700.001, 169080.005 518520.001, 168710.005 518940.001, 169010.005 519340.001, 169790.005 519780.001, 170260.005 519900.001, 170330.005 519560.001, 169820.005 519260.001, 169430.005 518700.001))",4,4,310296,PV24,Flevoland,1995PV24,provincie,6.755392e+05
1022,1022,Schokland,11189,1369,1853,"POLYGON ((181460.005 514400.001, 181200.005 514380.001, 181000.005 515200.001, 181100.005 516050.001, 181030.005 516750.001, 181050.005 517600.001, 181000.005 517950.001, 181250.005 518700.001, 181400.005 519000.001, 181600.005 519050.001, 181750.005 518720.001, 181550.005 517450.001, 181620.005 516200.001, 181540.005 515800.001, 181630.005 515300.001, 181460.005 514400.001))",Schokland,11189,1369,1853,"POLYGON ((181460.005 514400.001, 181200.005 514380.001, 181000.005 515200.001, 181100.005 516050.001, 181030.005 516750.001, 181050.005 517600.001, 181000.005 517950.001, 181250.005 518700.001, 181400.005 519000.001, 181600.005 519050.001, 181750.005 518720.001, 181550.005 517450.001, 181620.005 516200.001, 181540.005 515800.001, 181630.005 515300.001, 181460.005 514400.001))",4,4,310296,PV24,Flevoland,1995PV24,provincie,2.372100e+06


In [ ]:
# De plaatsen Schokland en Urk in de 1853 dataset zijn aan de nog niet bestaande provincie Flevoland toegekend... Deze is handmatig aangepast naar Overijssel (schokland) en Noord-Holland (Urk) (deze toekenning is gehaald uit het petitiebestand). (Idealiter hebben we een provinciegrenzen bestand. Of een CBS code icm provincie bestand.) 
result.loc[result["naam"].str.contains("Urk", na=False),"statnaam"] = "Noord-Holland"
result.loc[result["naam"].str.contains("Schokland", na=False),"statnaam"] = "Overijssel"

In [158]:
gem_1853_provincie = result[["naam", "cbscode", "geometry", "statnaam"]].rename(
    columns={"naam": "gemeente", "statnaam": "provincie"}
)

In [159]:
gem_1853_provincie.head()

,gemeente,cbscode,geometry,provincie
0,Sint Anna Termuiden,1177,"POLYGON ((15040.004 370729.999, 14490.004 369979.999, 13240.004 371519.999, 14950.004 373399.999, 15620.004 372619.999, 14990.004 371789.999, 15040.004 370729.999))",Zeeland
1,Heille,1073,"POLYGON ((17450.004 370869.999, 17620.004 370489.999, 17500.004 370239.999, 17790.004 368709.999, 18660.004 367449.999, 17920.004 366909.999, 18160.004 366379.999, 17670.004 365579.999, 17170.004 365579.999, 17070.004 365399.999, 16780.004 364889.999, 16470.004 364939.999, 16170.004 365509.999, 15540.004 365879.999, 15160.004 366859.999, 14610.004 366869.999, 14330.004 367179.999, 14410.004 367569.999, 14710.004 367639.999, 14720.004 368299.999, 14070.004 368819.999, 13790.004 369289.999, 13820.004 369789.999, 14490.004 369979.999, 15040.004 370729.999, 15850.004 370609.999, 16320.004 371229.999, 17100.004 371279.999, 17000.004 371009.999, 17450.004 370869.999))",Zeeland
2,Sluis,713,"POLYGON ((16320.004 371229.999, 15850.004 370609.999, 15040.004 370729.999, 14990.004 371789.999, 15620.004 372619.999, 15700.004 371969.999, 16320.004 371229.999))",Zeeland
3,Eede,1052,"POLYGON ((18930.004 366129.999, 18920.004 365399.999, 19530.004 365459.999, 20420.004 365379.999, 20710.004 366119.999, 21680.004 365599.999, 21770.004 364919.999, 24780.005 363749.999, 24830.005 363519.999, 19100.004 363049.999, 18220.004 363619.999, 17750.004 363469.999, 17580.004 364639.999, 17070.004 365399.999, 17170.004 365579.999, 17670.004 365579.999, 18160.004 366379.999, 18500.004 366149.999, 18930.004 366129.999))",Zeeland
4,Sint Kruis,1178,"POLYGON ((25090.005 369759.999, 25070.005 369319.999, 26550.005 368879.999, 26860.005 368559.999, 26120.005 368479.999, 25880.005 367889.999, 25420.005 367579.999, 25050.005 368159.999, 24090.005 367959.999, 24780.005 363749.999, 21770.004 364919.999, 21680.004 365599.999, 20710.004 366119.999, 20700.004 366539.999, 21060.004 366559.999, 21000.004 368629.999, 21560.004 369499.999, 22280.004 369369.999, 22330.004 369799.999, 22840.004 369839.999, 22890.004 370499.999, 23640.004 370629.999, 24290.004 369769.999, 25090.005 369759.999))",Zeeland


In [160]:
# het tabblad conversie is het te koppelen bestand en is gemaakt op basis van blad 1 uit het bestand.
pet_1853 = pd.read_excel(folder_path+"Woonplaatsenlijst_adres_1853.xlsx", sheet_name="conversie")

In [161]:
# id toegevoegd aan de petitie om dubbele matches er later uit te halen (gevallen als Bergen NH vs Bergen L etc. )
pet_1853['id'] = range(1, len(pet_1853) + 1)
pet_1853['id']= pet_1853['id'].astype(int)

In [162]:
pet_1853.head()

,Woonplaats,aantal name,Provincie,id
0,Adorp,48,Groningen,1
1,Aduard,60,Groningen,2
2,"Andel, den",66,Groningen,3
3,"Apel, Ter",36,Groningen,4
4,Appingedam,84,Groningen,5


In [ ]:
# inner join voor alle gevallen die volledig matches voor plaatsnaam en provincie
gem_1853_petitie_true = gem_1853_provincie.merge(
    pet_1853,
    left_on=["gemeente", "provincie"],
    right_on=["Woonplaats", "Provincie"],
    how="inner"
)

In [171]:
gem_1853_petitie_true.shape

(427, 8)

In [166]:
petitie_not_gem_1853 = gem_1853_provincie.merge(
    pet_1853,
    left_on=["gemeente", "provincie"],
    right_on=["Woonplaats", "Provincie"],
    how="right"
)

In [168]:
petitie_not_gem_1853_sub = petitie_not_gem_1853[petitie_not_gem_1853['gemeente'].isnull()]

In [169]:
petitie_not_gem_1853_sub.head()

,gemeente,cbscode,geometry,provincie,Woonplaats,aantal name,Provincie,id
2,NaN,NaN,None,NaN,"Andel, den",66,Groningen,3
3,NaN,NaN,None,NaN,"Apel, Ter",36,Groningen,4
9,NaN,NaN,None,NaN,"Boer, Ten",52,Groningen,10
12,NaN,NaN,None,NaN,Eenum,21,Groningen,13
13,NaN,NaN,None,NaN,Eexta,64,Groningen,14


In [170]:
petitie_not_gem_1853_sub = petitie_not_gem_1853_sub.drop(columns=["gemeente", "cbscode", "geometry", "provincie"])

In [175]:
petitie_not_gem_1853_sub.shape

(506, 4)

In [178]:
#geo = geocode_list_pdok_placeonly(df["raw_place"].tolist())

geocode = petitie_not_gem_1853_sub.join(geocode_list_pdok_placeonly(petitie_not_gem_1853_sub["Woonplaats"].tolist()).set_index("query"), on="Woonplaats")

In [184]:
geocode.shape

(508, 9)

In [183]:
geocode = geocode.drop_duplicates()

In [185]:
geocode

,Woonplaats,aantal name,Provincie,id,match_label,type,lat,lon,score
2,"Andel, den",66,Groningen,3,"Den Andel, Het Hogeland, Groningen",woonplaats,53.403824,6.518971,13.550007
3,"Apel, Ter",36,Groningen,4,"Ter Apel, Westerwolde, Groningen",woonplaats,52.882894,7.078466,15.637362
9,"Boer, Ten",52,Groningen,10,"Ten Boer, Groningen, Groningen",woonplaats,53.273014,6.694938,16.691298
12,Eenum,21,Groningen,13,"Eenum, Eemsdelta, Groningen",woonplaats,53.336641,6.783525,14.220003
13,Eexta,64,Groningen,14,None,None,NaN,NaN,NaN
14,Enumatil,50,Groningen,15,"Enumatil, Westerkwartier, Groningen",woonplaats,53.214580,6.410933,13.768633
16,Farnsum,53,Groningen,17,None,None,NaN,NaN,NaN
17,Garrelsweer,9,Groningen,18,"Garrelsweer, Eemsdelta, Groningen",woonplaats,53.305256,6.770773,13.276745
18,Garsthuizen,28,Groningen,19,"Garsthuizen, Eemsdelta, Groningen",woonplaats,53.372130,6.730324,13.923765
20,Grootegast en Oldekerk,31,Groningen,21,"Oldekerk, Westerkwartier, Groningen",woonplaats,53.229144,6.331670,12.803146


In [ ]:
geocode_matched = geocode[geocode["lat"].notna()]

In [ ]:
geocode_matched.shape

(422, 8)